In [1]:
import numpy as np
import pandas as pd
from google.colab import drive
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch
from torch.utils.data import Dataset, DataLoader
import pickle
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score,accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score, f1_score

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
main_dir = 'drive/MyDrive/23Spring-MLMA/isolated/'
os.listdir(main_dir)

['ReadMe',
 '002',
 '005',
 '004',
 '003',
 '001',
 '006',
 'RF1.pkl',
 'RF0.pkl',
 'RF2.pkl',
 'RF3.pkl',
 'RF4.pkl',
 'RF5.pkl',
 'RFt_patient_0model_1.pkl',
 'RFt_patient_0model_3.pkl',
 'RFt_patient_0model_5.pkl',
 'RFt_patient_0model_7.pkl',
 'RFt_patient_0model_9.pkl',
 'RFt_patient_1model_1.pkl',
 'RFt_patient_1model_3.pkl',
 'RFt_patient_1model_5.pkl',
 'RFt_patient_1model_7.pkl',
 'RFt_patient_1model_9.pkl',
 'RFt_patient_2model_1.pkl',
 'RFt_patient_2model_3.pkl',
 'RFt_patient_2model_5.pkl',
 'RFt_patient_2model_7.pkl',
 'RFt_patient_2model_9.pkl',
 'RFt_patient_3model_1.pkl',
 'RFt_patient_3model_3.pkl',
 'RFt_patient_3model_5.pkl',
 'RFt_patient_3model_7.pkl',
 'RFt_patient_3model_9.pkl',
 'RFt_patient_4model_1.pkl',
 'RFt_patient_4model_3.pkl',
 'RFt_patient_4model_5.pkl',
 'RFt_patient_4model_7.pkl',
 'RFt_patient_4model_9.pkl',
 'RFt_patient_5model_1.pkl',
 'RFt_patient_5model_3.pkl',
 'RFt_patient_5model_5.pkl',
 'RFt_patient_5model_7.pkl',
 'RFt_patient_5model_9.pkl',

# Split on patient level - User-independent


> 6-fold cross-validation





In [16]:
# get data
def get_original_feature_label(path):
    # scale both vertical and horizontal between 0 and 1
    scaler = MinMaxScaler()
    feature = []
    label = []
    for i in os.listdir(path):
        a = pd.read_csv(str(path+i),names=["vertical","horizontal"])
        a = np.array(a)
        a = scaler.fit_transform(a)
        a = np.swapaxes(a,0,1)
        feature.append(a)
        if i.endswith("001.csv"):
            label.append(0)
        elif i.endswith("002.csv"):
            label.append(1)
        elif i.endswith("003.csv"):
            label.append(2)
        elif i.endswith("004.csv"):
            label.append(3)
        elif i.endswith("005.csv"):
            label.append(4)
        elif i.endswith("006.csv"):
            label.append(5)
        elif i.endswith("007.csv"):
            label.append(6)
        elif i.endswith("008.csv"):
            label.append(7)
        elif i.endswith("009.csv"):
            label.append(8)
        elif i.endswith("010.csv"):
            label.append(9)
        elif i.endswith("011.csv"):
            label.append(10)
        elif i.endswith("012.csv"):
            label.append(11)
    
    return feature,label

In [18]:
from sklearn.ensemble import RandomForestClassifier
def train(train_feature,train_label,test_feature,test_label,num):
    # 定义模型
    model = RandomForestClassifier(n_estimators=100, random_state=42)

    # 训练模型
    model.fit(train_feature, train_label)#,eval_set=[(test_feature, test_label)])

    # 预测测试集
    y_pred_p = model.predict_proba(test_feature)
    y_pred = model.predict(test_feature)

    # 计算准确率
    accuracy = (y_pred == test_label).sum() / len(test_label)
    auc = roc_auc_score(test_label, y_pred_p, multi_class='ovr')
    average_method = 'macro' # 'macro' 或 'micro' 或 'weighted'
    f1 = f1_score(test_label, y_pred,average=average_method)#, multi_class='ovr')
    print('Accuracy:', accuracy)
    print('AUC:', auc)
    print('F1:', f1)

    # 保存模型
    pickle.dump(model, open(main_dir+"RF"+str(num)+".pkl", 'wb'))

    return accuracy,auc,f1

In [19]:
patient_all = ["001",'002','003','004','005','006']
accl = [];aucl = []; f1l = []
for j in range(6):
  patient = [patient_all[j]]
  test_feature = []
  test_label = []
  for i in patient:
      f,l = get_original_feature_label(str(main_dir+i+"/isolated_strokes/"))
      for x in f:
          test_feature.append(x)         
      for x in l:
          test_label.append(x)
  print(patient)
  print(len(test_feature))
  test_feature = np.reshape(test_feature,(np.shape(test_feature)[0],-1))

  patient = [x for x in patient_all if x != patient_all[j]]
  print(patient)
  train_feature = []
  train_label = []
  for i in patient:
      f,l = get_original_feature_label(str(main_dir+i+"/isolated_strokes/"))
      for x in f:
          train_feature.append(x)         
      for x in l:
          train_label.append(x)
  print(len(train_feature))
  train_feature = np.reshape(train_feature,(np.shape(train_feature)[0],-1))
  acc, auc, f1 = train(train_feature,train_label,test_feature,test_label,j)
  accl.append(acc)
  aucl.append(auc)
  f1l.append(f1)
  # print(acc)

['001']
120
['002', '003', '004', '005', '006']
604
Accuracy: 0.6416666666666667
AUC: 0.9297727272727273
F1: 0.6194706029264853
['002']
120
['001', '003', '004', '005', '006']
604
Accuracy: 0.8583333333333333
AUC: 0.9960606060606061
F1: 0.8580665865495587
['003']
122
['001', '002', '004', '005', '006']
602
Accuracy: 0.5655737704918032
AUC: 0.9523250726375726
F1: 0.5467135188102007
['004']
121
['001', '002', '003', '005', '006']
603
Accuracy: 0.6115702479338843
AUC: 0.9181740624922443
F1: 0.5968008516537929
['005']
120
['001', '002', '003', '004', '006']
604
Accuracy: 0.7
AUC: 0.974225048651654
F1: 0.6806288629787195
['006']
121
['001', '002', '003', '004', '005']
603
Accuracy: 0.7851239669421488
AUC: 0.9753750651477925
F1: 0.7685583071612484


In [20]:
np.mean(aucl)

0.9576554303770995

In [7]:
print(np.mean(accl))

0.6937113308946395


In [8]:
print(np.mean(f1l))

0.6783731216800009


# User-dependent


> 5-fold cross-validation





In [21]:
# get data
def get_original_feature_label(path, t1, t2):
    # scale both vertical and horizontal between 0 and 1
    scaler = MinMaxScaler()
    feature = []
    label = []
    if t2==0:
        num = len(os.listdir(path))-2
    else:
        num = 2
    for i in os.listdir(path):
        a = pd.read_csv(str(path+i),names=["vertical","horizontal"])
        a = np.array(a)
        a = scaler.fit_transform(a)
        a = np.swapaxes(a,0,1)
        if (t1==1)&(t2!=0)&((i[8:10]=="01") or (i[8:10]=="02")):
            feature.append(a)
            if i.endswith("001.csv"):
                label.append(0)
            elif i.endswith("002.csv"):
                label.append(1)
            elif i.endswith("003.csv"):
                label.append(2)
            elif i.endswith("004.csv"):
                label.append(3)
            elif i.endswith("005.csv"):
                label.append(4)
            elif i.endswith("006.csv"):
                label.append(5)
            elif i.endswith("007.csv"):
                label.append(6)
            elif i.endswith("008.csv"):
                label.append(7)
            elif i.endswith("009.csv"):
                label.append(8)
            elif i.endswith("010.csv"):
                label.append(9)
            elif i.endswith("011.csv"):
                label.append(10)
            elif i.endswith("012.csv"):
                label.append(11)
            continue
        if (t1==3)&(t2!=0)&((i[8:10]=="03") or (i[8:10]=="04")):
            feature.append(a)
            if i.endswith("001.csv"):
                label.append(0)
            elif i.endswith("002.csv"):
                label.append(1)
            elif i.endswith("003.csv"):
                label.append(2)
            elif i.endswith("004.csv"):
                label.append(3)
            elif i.endswith("005.csv"):
                label.append(4)
            elif i.endswith("006.csv"):
                label.append(5)
            elif i.endswith("007.csv"):
                label.append(6)
            elif i.endswith("008.csv"):
                label.append(7)
            elif i.endswith("009.csv"):
                label.append(8)
            elif i.endswith("010.csv"):
                label.append(9)
            elif i.endswith("011.csv"):
                label.append(10)
            elif i.endswith("012.csv"):
                label.append(11)
            continue
        if (t1==5)&(t2!=0)&((i[8:10]=="05") or (i[8:10]=="06")):
            feature.append(a)
            if i.endswith("001.csv"):
                label.append(0)
            elif i.endswith("002.csv"):
                label.append(1)
            elif i.endswith("003.csv"):
                label.append(2)
            elif i.endswith("004.csv"):
                label.append(3)
            elif i.endswith("005.csv"):
                label.append(4)
            elif i.endswith("006.csv"):
                label.append(5)
            elif i.endswith("007.csv"):
                label.append(6)
            elif i.endswith("008.csv"):
                label.append(7)
            elif i.endswith("009.csv"):
                label.append(8)
            elif i.endswith("010.csv"):
                label.append(9)
            elif i.endswith("011.csv"):
                label.append(10)
            elif i.endswith("012.csv"):
                label.append(11)
            continue
        if (t1==7)&(t2!=0)&((i[8:10]=="07") or (i[8:10]=="08")):
            feature.append(a)
            if i.endswith("001.csv"):
                label.append(0)
            elif i.endswith("002.csv"):
                label.append(1)
            elif i.endswith("003.csv"):
                label.append(2)
            elif i.endswith("004.csv"):
                label.append(3)
            elif i.endswith("005.csv"):
                label.append(4)
            elif i.endswith("006.csv"):
                label.append(5)
            elif i.endswith("007.csv"):
                label.append(6)
            elif i.endswith("008.csv"):
                label.append(7)
            elif i.endswith("009.csv"):
                label.append(8)
            elif i.endswith("010.csv"):
                label.append(9)
            elif i.endswith("011.csv"):
                label.append(10)
            elif i.endswith("012.csv"):
                label.append(11)
            continue
        if (t1==9)&(t2!=0)&((i[8:10]=="09") or (i[8:10]=="10")):
            feature.append(a)
            if i.endswith("001.csv"):
                label.append(0)
            elif i.endswith("002.csv"):
                label.append(1)
            elif i.endswith("003.csv"):
                label.append(2)
            elif i.endswith("004.csv"):
                label.append(3)
            elif i.endswith("005.csv"):
                label.append(4)
            elif i.endswith("006.csv"):
                label.append(5)
            elif i.endswith("007.csv"):
                label.append(6)
            elif i.endswith("008.csv"):
                label.append(7)
            elif i.endswith("009.csv"):
                label.append(8)
            elif i.endswith("010.csv"):
                label.append(9)
            elif i.endswith("011.csv"):
                label.append(10)
            elif i.endswith("012.csv"):
                label.append(11)
            continue

        if (t1==1)&(t2==0)&((i[8:10]=="01") or (i[8:10]=="02")):
           continue
        if (t1==3)&(t2==0)&((i[8:10]=="03") or (i[8:10]=="04")):
           continue
        if (t1==5)&(t2==0)&((i[8:10]=="05") or (i[8:10]=="06")):
           continue
        if (t1==7)&(t2==0)&((i[8:10]=="07") or (i[8:10]=="08")):
           continue
        if (t1==9)&(t2==0)&((i[8:10]=="09") or (i[8:10]=="10")):
           continue

        if (t2==0):
          feature.append(a)
          if i.endswith("001.csv"):
              label.append(0)
          elif i.endswith("002.csv"):
              label.append(1)
          elif i.endswith("003.csv"):
              label.append(2)
          elif i.endswith("004.csv"):
              label.append(3)
          elif i.endswith("005.csv"):
              label.append(4)
          elif i.endswith("006.csv"):
              label.append(5)
          elif i.endswith("007.csv"):
              label.append(6)
          elif i.endswith("008.csv"):
              label.append(7)
          elif i.endswith("009.csv"):
              label.append(8)
          elif i.endswith("010.csv"):
              label.append(9)
          elif i.endswith("011.csv"):
              label.append(10)
          elif i.endswith("012.csv"):
              label.append(11)
        
    
    return feature,label

In [22]:
from sklearn.ensemble import RandomForestClassifier
def train(train_feature,train_label,test_feature,test_label,num,p):
    # 定义模型
    model = RandomForestClassifier(n_estimators=100, random_state=42)

    # 训练模型
    model.fit(train_feature, train_label)#,eval_set=[(test_feature, test_label)])

    # 预测测试集
    y_pred_p = model.predict_proba(test_feature)
    y_pred = model.predict(test_feature)

    # 计算准确率
    accuracy = (y_pred == test_label).sum() / len(test_label)
    auc = roc_auc_score(test_label, y_pred_p, multi_class='ovr')
    average_method = 'macro' # 'macro' 或 'micro' 或 'weighted'
    f1 = f1_score(test_label, y_pred,average=average_method)#, multi_class='ovr')
    print('Accuracy:', accuracy)
    print('AUC:', auc)
    print('F1:', f1)

    # 保存模型
    pickle.dump(model, open(main_dir+"RFt_patient_"+str(p)+"model_"+str(num)+".pkl", 'wb'))

    return accuracy,auc,f1

In [23]:
patient_all = ["001",'002','003','004','005','006']
acc_p = []; auc_p = []; f1_p = []
for j in range(6):

  patient_model = patient_all[j]
  patient_path = str(main_dir+patient_model+"/isolated_strokes/")
  accl = []; aucl = []; f1l = []

  for i in range(1,10,2):
    train_feature = []
    train_label = []
    f,l = get_original_feature_label(patient_path,i,0)
    for x in f:
        train_feature.append(x)         
    for x in l:
        train_label.append(x)
    print(len(train_label))

    test_feature = []
    test_label = []
    f,l = get_original_feature_label(patient_path,i,1)
    for x in f:
        test_feature.append(x)         
    for x in l:
        test_label.append(x)
    print(len(test_label))
    
    test_feature = np.reshape(test_feature,(np.shape(test_feature)[0],-1))
    train_feature = np.reshape(train_feature,(np.shape(train_feature)[0],-1))
    acc,auc,f1 = train(train_feature,train_label,test_feature,test_label,i,j)
    accl.append(acc)
    aucl.append(auc)
    f1l.append(f1)
    # print(acc)
  acc_p.append(np.mean(accl))
  auc_p.append(np.mean(aucl))
  f1_p.append(np.mean(f1l))

96
24
Accuracy: 0.7916666666666666
AUC: 0.9905303030303031
F1: 0.7861111111111111
96
24
Accuracy: 0.9583333333333334
AUC: 1.0
F1: 0.9555555555555556
96
24
Accuracy: 0.7083333333333334
AUC: 0.9753787878787877
F1: 0.6861111111111112
96
24
Accuracy: 0.9583333333333334
AUC: 0.9962121212121211
F1: 0.9555555555555556
96
24
Accuracy: 0.9583333333333334
AUC: 0.9924242424242423
F1: 0.9555555555555556
96
24
Accuracy: 0.9583333333333334
AUC: 0.9990530303030303
F1: 0.9555555555555556
96
24
Accuracy: 0.8333333333333334
AUC: 0.9943181818181818
F1: 0.8333333333333331
96
24
Accuracy: 0.9583333333333334
AUC: 1.0
F1: 0.9555555555555556
96
24
Accuracy: 1.0
AUC: 1.0
F1: 1.0
96
24
Accuracy: 0.9166666666666666
AUC: 0.9981060606060606
F1: 0.9138888888888889
98
24
Accuracy: 0.75
AUC: 0.9810606060606061
F1: 0.7111111111111111
98
24
Accuracy: 0.8333333333333334
AUC: 0.975378787878788
F1: 0.8333333333333334
98
24
Accuracy: 0.9166666666666666
AUC: 0.9990530303030303
F1: 0.9111111111111111
98
24
Accuracy: 0.833333

In [12]:
acc_p

[0.875, 0.9333333333333333, 0.825, 0.75, 0.8659420289855072, 0.9]

In [13]:
np.mean(acc_p)

0.8582125603864735

In [14]:
np.mean(f1_p)

0.845707671957672

In [24]:
np.mean(auc_p)

0.9842051467051468